In [1]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd


import matplotlib.pyplot as plt

#from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import backend as K

import tensorflow as tf

# # Set some parameters
# IMG_WIDTH = 128
# IMG_HEIGHT = 128
# IMG_CHANNELS = 3
# TRAIN_PATH = '../input/stage1_train/'
# TEST_PATH = '../input/stage1_test/'

# warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
# seed = 42
# random.seed = seed
# np.random.seed = seed

Using TensorFlow backend.


In [2]:
img_height=128
img_width=128
border=5
path_train_input = 'C:/Users/Priyam/Desktop/Ag-Net-Dataset/input/'
path_train_output = 'C:/Users/Priyam/Desktop/Ag-Net-Dataset/target/'

images_path = next(os.walk(path_train_input))[2]
print(images_path)
images=np.asarray(images_path)
#print(images)
print(images[0][3:-6])

['lc8_2015245_10_11_1.tif', 'lc8_2015245_10_11_2.tif', 'lc8_2015245_10_11_3.tif', 'lc8_2015245_10_11_4.tif', 'lc8_2015245_10_11_5.tif', 'lc8_2015245_10_11_6.tif', 'lc8_2015245_10_11_7.tif', 'lc8_2015245_10_12_1.tif', 'lc8_2015245_10_12_2.tif', 'lc8_2015245_10_12_3.tif', 'lc8_2015245_10_12_4.tif', 'lc8_2015245_10_12_5.tif', 'lc8_2015245_10_12_6.tif', 'lc8_2015245_10_12_7.tif', 'lc8_2015245_10_13_1.tif', 'lc8_2015245_10_13_2.tif', 'lc8_2015245_10_13_3.tif', 'lc8_2015245_10_13_4.tif', 'lc8_2015245_10_13_5.tif', 'lc8_2015245_10_13_6.tif', 'lc8_2015245_10_13_7.tif', 'lc8_2015245_10_14_1.tif', 'lc8_2015245_10_14_2.tif', 'lc8_2015245_10_14_3.tif', 'lc8_2015245_10_14_4.tif', 'lc8_2015245_10_14_5.tif', 'lc8_2015245_10_14_6.tif', 'lc8_2015245_10_14_7.tif', 'lc8_2015245_10_15_1.tif', 'lc8_2015245_10_15_2.tif', 'lc8_2015245_10_15_3.tif', 'lc8_2015245_10_15_4.tif', 'lc8_2015245_10_15_5.tif', 'lc8_2015245_10_15_6.tif', 'lc8_2015245_10_15_7.tif', 'lc8_2015245_10_16_1.tif', 'lc8_2015245_10_16_2.tif', 

In [3]:
lst=[]
j=1
temp=[]

for image in images:
    i=(j%7)+1
    if(j%7!=0):
        img = load_img(path_train_input + 'lc8' + image[3:-6] + '_' + str(i) + '.tif', grayscale=True)
        img = img_to_array(img)
        img=np.array(img) 
        img=np.reshape(img, (16384,1))
        #print(img.shape)
        temp.append(img)
        #lst.append(img)
        j+=1
    else:
        img = load_img(path_train_input + 'lc8' + image[3:-6] + '_' + str(i) + '.tif', grayscale=True)
        img = img_to_array(img)
        img=np.array(img) 
        img=np.reshape(img, (16384,1))
        #print(img.shape)
        temp.append(img)
        #lst.append(img)
        j+=1
        
        temp=np.array(temp)
        temp=np.reshape(temp,(128,128,7))
        lst.append(temp)
        temp=[]
        
        
        
        
            

lst=np.array(lst)    
print(lst.shape)
#print(lst[0])
#for v in img[0]:
 #   print((v))



C:\Users\Priyam\Anaconda3\lib\site-packages\keras_preprocessing\image\utils.py:98: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


(1593, 128, 128, 7)


In [4]:
print((lst[0]).shape)
x=lst

(128, 128, 7)


In [6]:
images_path = next(os.walk(path_train_output))[2]
#print(images_path)
images=np.asarray(images_path)
print(images[1])
print(images[1][3:-4])
lst=[]

for image in images:
    
    
    img = load_img(path_train_output + 'cdl' + image[3:-4]  + '.tif', grayscale=True)
    img = img_to_array(img)
    img=img/255
    #img=np.array(img) 
    #img=np.reshape(img, (16384,1))
    #print(img.shape)
    lst.append(img)
    #lst.append(img)
       
        
        
        
            

lst=np.array(lst)    
print(lst.shape)
#print(lst[0])
#for v in img[0]:
 #   print((v))



cdl_2015245_10_12.tif
_2015245_10_12
(1593, 128, 128, 1)


In [7]:
y=lst[:1593]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [9]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [10]:
inputs= Input((128,128,7))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
model.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 7)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 128, 128, 7)  0           input_1[0][0]                    
________________________________________________________________________________________________

In [ ]:
earlystopper = EarlyStopping(patience=500, verbose=1)
checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
results = model.fit(X_train, y_train, validation_split=0.1, batch_size=16, epochs=500, 
                    callbacks=[earlystopper, checkpointer])

Train on 1146 samples, validate on 128 samples
Epoch 1/500
1146/1146 [==============================] - 158s 138ms/step - loss: 0.6384 - mean_iou: 0.4755 - val_loss: 0.6104 - val_mean_iou: 0.4957

Epoch 00001: val_loss improved from inf to 0.61044, saving model to model-dsbowl2018-1.h5
Epoch 2/500
1146/1146 [==============================] - 153s 133ms/step - loss: 0.6194 - mean_iou: 0.4971 - val_loss: 0.6122 - val_mean_iou: 0.4979

Epoch 00002: val_loss did not improve from 0.61044
Epoch 3/500
1146/1146 [==============================] - 154s 134ms/step - loss: 0.6176 - mean_iou: 0.4983 - val_loss: 0.6103 - val_mean_iou: 0.4986

Epoch 00003: val_loss improved from 0.61044 to 0.61025, saving model to model-dsbowl2018-1.h5
Epoch 4/500
1146/1146 [==============================] - 158s 138ms/step - loss: 0.6159 - mean_iou: 0.4988 - val_loss: 0.6107 - val_mean_iou: 0.4990

Epoch 00004: val_loss did not improve from 0.61025
Epoch 5/500
1146/1146 [==============================] - 153s 134ms

1146/1146 [==============================] - 162s 141ms/step - loss: 0.6148 - mean_iou: 0.4999 - val_loss: 0.6096 - val_mean_iou: 0.4999

Epoch 00041: val_loss did not improve from 0.60954
Epoch 42/500
1146/1146 [==============================] - 154s 134ms/step - loss: 0.6147 - mean_iou: 0.4999 - val_loss: 0.6095 - val_mean_iou: 0.4999

Epoch 00042: val_loss improved from 0.60954 to 0.60953, saving model to model-dsbowl2018-1.h5
Epoch 43/500
1146/1146 [==============================] - 154s 134ms/step - loss: 0.6152 - mean_iou: 0.4999 - val_loss: 0.6102 - val_mean_iou: 0.4999

Epoch 00043: val_loss did not improve from 0.60953
Epoch 44/500
1146/1146 [==============================] - 153s 134ms/step - loss: 0.6148 - mean_iou: 0.4999 - val_loss: 0.6100 - val_mean_iou: 0.4999

Epoch 00044: val_loss did not improve from 0.60953
Epoch 45/500
1146/1146 [==============================] - 153s 134ms/step - loss: 0.6148 - mean_iou: 0.4999 - val_loss: 0.6098 - val_mean_iou: 0.4999

Epoch 00045


Epoch 00080: val_loss did not improve from 0.60947
Epoch 81/500
1146/1146 [==============================] - 154s 134ms/step - loss: 0.6149 - mean_iou: 0.4999 - val_loss: 0.6102 - val_mean_iou: 0.4999

Epoch 00081: val_loss did not improve from 0.60947
Epoch 82/500
1146/1146 [==============================] - 153s 134ms/step - loss: 0.6150 - mean_iou: 0.4999 - val_loss: 0.6109 - val_mean_iou: 0.4999

Epoch 00082: val_loss did not improve from 0.60947
Epoch 83/500
1146/1146 [==============================] - 153s 133ms/step - loss: 0.6150 - mean_iou: 0.4999 - val_loss: 0.6100 - val_mean_iou: 0.4999

Epoch 00083: val_loss did not improve from 0.60947
Epoch 84/500
1146/1146 [==============================] - 153s 133ms/step - loss: 0.6149 - mean_iou: 0.4999 - val_loss: 0.6104 - val_mean_iou: 0.4999

Epoch 00084: val_loss did not improve from 0.60947
Epoch 85/500
1146/1146 [==============================] - 154s 134ms/step - loss: 0.6151 - mean_iou: 0.4999 - val_loss: 0.6099 - val_mean_io

1146/1146 [==============================] - 160s 140ms/step - loss: 0.6149 - mean_iou: 0.5000 - val_loss: 0.6099 - val_mean_iou: 0.5000

Epoch 00121: val_loss did not improve from 0.60947
Epoch 122/500
1146/1146 [==============================] - 161s 141ms/step - loss: 0.6149 - mean_iou: 0.5000 - val_loss: 0.6100 - val_mean_iou: 0.5000

Epoch 00122: val_loss did not improve from 0.60947
Epoch 123/500
1146/1146 [==============================] - 156s 136ms/step - loss: 0.6148 - mean_iou: 0.5000 - val_loss: 0.6100 - val_mean_iou: 0.5000

Epoch 00123: val_loss did not improve from 0.60947
Epoch 124/500
1146/1146 [==============================] - 158s 138ms/step - loss: 0.6149 - mean_iou: 0.5000 - val_loss: 0.6100 - val_mean_iou: 0.5000

Epoch 00124: val_loss did not improve from 0.60947
Epoch 125/500
1146/1146 [==============================] - 159s 139ms/step - loss: 0.6148 - mean_iou: 0.5000 - val_loss: 0.6099 - val_mean_iou: 0.5000

Epoch 00125: val_loss did not improve from 0.60947

In [ ]:
model = load_model('model-dsbowl2018-1.h5', custom_objects={'mean_iou': mean_iou})
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

In [ ]:
# Perform a sanity check on some random training samples
ix = random.randint(0, len(preds_train_t))
imshow(X_train[ix])
plt.show()
imshow(np.squeeze(Y_train[ix]))
plt.show()
imshow(np.squeeze(preds_train_t[ix]))
plt.show()

In [ ]:
# Perform a sanity check on some random validation samples
ix = random.randint(0, len(preds_val_t))
imshow(X_train[int(X_train.shape[0]*0.9):][ix])
plt.show()
imshow(np.squeeze(Y_train[int(Y_train.shape[0]*0.9):][ix]))
plt.show()
imshow(np.squeeze(preds_val_t[ix]))
plt.show()

In [ ]:

    

model = get_unet(inputs)

model.compile(optimizer=Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [ ]:
results = model.fit(X_train, y_train, batch_size=256, epochs=100, callbacks=callbacks,
                    validation_data=(X_valid, y_valid))

In [ ]:



model_json = model.to_json()
with open("D:/All/sat_crop/Ag-Net-Dataset/model.json", "w") as json_file: #Check the folder path once :NOTE
    json_file.write(model_json)

model.save_weights("D:/All/sat_crop/Ag-Net-Dataset/model.h5") #Check the folder path once :NOTE
print("Saved model to disk")


json_file = open('D:/All/VipulSirProject/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)


loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


# load weights into new model
loaded_model.load_weights("D:/All/VipulSirProject/model.h5")
print("Loaded model from disk")
